# Solution Outline:
1. For each sample, create augmented vectors.
2. For each sample, extract statistical information from all created vectors; these are the features
3. Use an Ensemble Learner composed of 3 classifiers with different strengths and weaknesses

# Notebook Guidelines:
* Wrap everything in functions to have as many forgotten variables garbage collected
* Write `typing` hints to express our intentions better
* Serialize expensive computations with the joblib decorator
* Try and reinvent the wheel as less as possible; it's probably already implemented in `sklearn`

In [1]:
import gc
gc.collect()

0

In [2]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import typing
from typing import List, Union, Optional, Tuple, Iterator
from multiprocessing import Pool
from sklearn.externals.joblib import Memory

from scipy import signal

In [3]:
NUM_USERS = 20
NUM_TRAIN_OBSERVATIONS = 450
NAMED_COLUMNS = ['x', 'y', 'z']
SAMPLE_RATE = 100
DELTA = 1.5
TOTAL_SAMPLES = NUM_USERS * NUM_TRAIN_OBSERVATIONS
SEED = 12855218
NUM_QUERIES = 5000
DATA_SPLIT = 1

INPUT_PATH = '../input/'
TEST_PATH = INPUT_PATH + 'test/test/'
TRAIN_PATH = INPUT_PATH + 'train/train/'

In [4]:
job_cache = Memory(location='./cache', verbose=0)

In [5]:
def clear_cache():
    !rm -rf ./cache
#clear_cache()

In [6]:
def load_data(labels: pd.DataFrame, folder_path: str) -> Union[List[pd.DataFrame], List[int]]:
    result = []
    classes = []
    for _, row in labels.iterrows():
        csv_id = row['id']
        user = row['class'] - 1
        classes.append(user)
        result.append(pd.read_csv(folder_path + str(csv_id) + '.csv', names=NAMED_COLUMNS))
    return result, classes

In [7]:
test_data, test_classes = load_data(pd.read_csv(INPUT_PATH + 'sample_submission.csv'), TEST_PATH)

In [8]:
train_data, train_classes = load_data(pd.read_csv(INPUT_PATH + 'train_labels.csv'), TRAIN_PATH)

## Augmented Vectors
Various signal-related stuff happens here. Creating a mix of the features from the [Human Activity Recognition Using Smartphones](https://github.com/anas337/Human-Activity-Recognition-Using-Smartphones/blob/master/Data/Original-Data/HAPT-Dataset/Processed-Data/features_info.md) dataset and what I found interesting from [tsfresh](https://tsfresh.readthedocs.io/en/latest/text/list_of_features.html) or other Kaggle competitions. Noise elimination is done by a Butterworth filter, whose paramaters were chosen experimentally by visualising how much it cancels.

In [9]:
 class Observation:
    def butter_lowpass(cutoff, nyq_freq, order=4):
        normal_cutoff = float(cutoff) / nyq_freq
        b, a = signal.butter(order, normal_cutoff, btype='lowpass')
        return b, a


    def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order):
        b, a = Observation.butter_lowpass(cutoff_freq, nyq_freq, order=order)
        y = signal.filtfilt(b, a, data)
        return y


    def filter_noise(x):
        BUTTER_CUTOFF = 40
        return Observation.butter_lowpass_filter(signal.medfilt(x), BUTTER_CUTOFF, 
                                                 SAMPLE_RATE / 2, order=3)


    def filter_body_acceleration(x):
        BUTTER_CUTOFF = 0.5
        y = Observation.butter_lowpass_filter(x, BUTTER_CUTOFF, SAMPLE_RATE / 2, order=3)
        return y, np.array(x) - np.array(y)


    def derive(x):
        return np.diff(x) / DELTA


    def compute_magnitude(a, b, c):
        return [np.linalg.norm(vector) for vector in zip(a, b, c)]


    def fft(x):
        return pd.Series(abs(np.fft.rfft(x)))


    def __init__(self, x, y, z):
        # x, y, z filtered
        self.data = pd.DataFrame()
        self.data['x'] = Observation.filter_noise(x)
        self.data['y'] = Observation.filter_noise(y)
        self.data['z'] = Observation.filter_noise(z)

        # body, acceleration
        for c in NAMED_COLUMNS:
            self.data['body_' + c], self.data['acc_' + c] = \
                Observation.filter_body_acceleration(self.data[c])
            self.data['jerk_' + c] = \
                pd.Series(Observation.derive(self.data['body_' + c]))
        for t in ['body', 'jerk']:
            # magnitudes
            args = [self.data[t + '_' + c] for c in NAMED_COLUMNS]
            self.data[t + '_magnitude'] = Observation.compute_magnitude(*args)
            # fourier transforms
            for c in NAMED_COLUMNS:
                self.data['f_' + t + '_' + c] = Observation.fft(self.data[t + '_' + c])
                self.data['f_' + t + '_magnitude'] = \
                    Observation.fft(self.data[t + '_magnitude'])

## Feature Extraction
I associate each vector obtain from the last step with a level, representing how many features I want to extract from it. These are various statistical informations and I have yet to perform any feature selection on them. My initial classfier, xgboost, had lots of trouble with classes $2$ and $15$ and adding sliding window statistics improved my accuracy for those.

In [10]:
class FeaturesExtractor:
    def bands_energy(a, time_step=1.0/64):
        fft = np.fft.rfft(a)
        f = np.fft.rfftfreq(a.size, d=time_step)
        mask = np.logical_and(np.abs(f) >= 5, np.abs(f) <= 15)
        return np.sum((np.abs(fft[mask]) ** 2) / a.size)


    def calc_change_rate(a):
        change = (np.diff(a) / a[:-1]).values
        change = change[np.nonzero(change)[0]]
        change = change[~np.isnan(change)]
        change = change[change != -np.inf]
        change = change[change != np.inf]
        return np.mean(change)


    def hann_mean(a):
        from scipy.signal import hann, convolve
        return (convolve(a, hann(10), mode='same') / sum(hann(10))).mean()


    def hilbert_mean(a):
        from scipy.signal import hilbert
        return np.abs(hilbert(a)).mean()


    def add_trend_feature(a, apply_abs=False):
        from sklearn.linear_model import LinearRegression
        idx = np.arange(len(a))
        lr = LinearRegression()
        lr.fit(idx.reshape(-1, 1), a if not apply_abs else np.abs(a))
        return lr.coef_[0]


    def entropy(a):
        import scipy.stats
        return scipy.stats.entropy(a)


    def trim_mean(a):
        from scipy.stats import trim_mean
        return trim_mean(a, 0.1)


    def extract(self, a, idx, level):
        def assign(s, v):
            self.f[idx + '_' + s] = pd.Series(v)

            
        a.dropna()
        if a.size == 0:
            return

        valid_level = lambda x: level >= x

        if not valid_level(0):
            return

        # all
        assign('last', a.values[-1])
        assign('min', a.min())
        assign('max', a.max())
        assign('std', a.std())
        assign('mean', a.mean())
        assign('mad', a.mad())
        assign('sum', a.sum())
        assign('energy', a.pow(2).sum() / a.count())

        if not valid_level(1):
            return

        # fft, x, base, jerk
        assign('iqr', np.subtract(*np.percentile(a, [75, 25])))
        assign('entropy', FeaturesExtractor.entropy(a))
        #assign('ar', arburg(a, 2)[1]) 
        assign('argmax', a.idxmax())
        assign('kurtosis', a.kurtosis())
        assign('skew', a.skew())
        assign('bands_energy', FeaturesExtractor.bands_energy(a))
        assign('q95', np.quantile(a, 0.95))
        assign('q99', np.quantile(a, 0.99))
        assign('q05', np.quantile(a, 0.05))
        assign('q01', np.quantile(a, 0.01))

        if not valid_level(2):
            return

        # x, base, jerk
        assign('mean_change_abs', np.mean(np.diff(a)))
        assign('mean_change_rate', FeaturesExtractor.calc_change_rate(a))
        assign('abs_max', np.abs(a).max())
        assign('abs_min', np.abs(a).min())
        assign('abs_mean', np.abs(a).mean())
        assign('abs_std', np.abs(a).std())
        assign('max_to_min', a.max() / np.abs(a.min()))
        assign('max_to_min_diff', a.max() - np.abs(a.min()))
        assign('med', a.median())

        if not valid_level(3):
            return

        # x
        assign('hann', FeaturesExtractor.hann_mean(a))
        assign('trend_feature', FeaturesExtractor.add_trend_feature(a))
        assign('abs_trend_feature', FeaturesExtractor.add_trend_feature(a, True))
        assign('hilbert_mean', FeaturesExtractor.hilbert_mean(a))
        assign('ave10', FeaturesExtractor.trim_mean(a))

        assign('rolling_mean', a.rolling(window=10).mean().mean(skipna=True))
        assign('exp_moving_average_10_mean', 
               (pd.Series.ewm(a, span=10).mean()).mean(skipna=True))
        assign('exp_moving_average_30_mean',
               (pd.Series.ewm(a, span=30).mean()).mean(skipna=True))
        for window_size in [5, 20, 50]:
            self.extract(a.rolling(window_size).std().dropna(), 
                         idx + '_window_std_' + str(window_size),
                         0)
            self.extract(a.rolling(window_size).mean().dropna(),
                         idx + '_window_mean_' + str(window_size),
                         0)


    def __init__(self, df):
        self.f = pd.DataFrame()
        add = lambda x, y: self.extract(df[x], x, y)
        for c in NAMED_COLUMNS:
            add(c, 3)
            for y in ['body_', 'jerk_']:
                add(y + c, 2)
                add('f_' + y + c, 1)

        for y in ['body', 'jerk']:
            add(y + '_magnitude', 1)
            add('f_' + y + '_magnitude', 1)

In [11]:
def extract_user_features(x: pd.DataFrame) -> pd.DataFrame:
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        return FeaturesExtractor(Observation(*[x[c] for c in NAMED_COLUMNS]).data).f

In [12]:
def extract_features(data: List[pd.DataFrame]) -> pd.DataFrame:
    with Pool() as pool:
        results = pd.concat(pool.map(extract_user_features, data))
        results.index = range(len(results))
        return results

In [13]:
test_features = extract_features(test_data)

Takes roughly $30$ minutes to extract the features from the train data on $4$ cores.

In [14]:
train_features = extract_features(train_data)

In [15]:
def join_train_test_features(train: pd.DataFrame, test: pd.DataFrame) -> pd.DataFrame:
    changed_tests = test.copy()
    changed_tests.index = changed_tests.index + TOTAL_SAMPLES
    return pd.concat([train, changed_tests])

In [16]:
def drop_nan_features(features: pd.DataFrame) -> pd.DataFrame:
    result = features.copy()
    result[~np.isfinite(result)] = np.nan
    return result.dropna(axis=1)

In [17]:
def split_train_test_features(joined_features: pd.DataFrame) -> Union[pd.DataFrame, pd.DataFrame]:
    train = joined_features.iloc[:TOTAL_SAMPLES]
    test = joined_features.iloc[TOTAL_SAMPLES:]
    test.index = test.index - TOTAL_SAMPLES
    return train, test

In [18]:
def filter_features(train: pd.DataFrame, test: pd.DataFrame) -> Union[pd.DataFrame, pd.DataFrame]:
    return split_train_test_features(drop_nan_features(join_train_test_features(train, test)))

At this point we drop $\approx 100$ columns out of the $\approx 550$ we originally had, because they contain infinite values or NaNs. In the other versions I imputed them with and had better results.

In [19]:
filtered_train_features, filtered_test_features = filter_features(train_features, test_features)

In [20]:
NUM_FEATURES = filtered_train_features.shape[1]

In [21]:
def get_dataset(df: pd.DataFrame, classes: List[int]) -> Union[np.ndarray, np.ndarray]:
    return df.values, np.array(classes)

In [23]:
x_train, y_train = get_dataset(filtered_train_features, train_classes)

## Classifier
* Ensemble learner: a voting classifier composed of an XGB, a RF and an SVC
* They take votes with weights chosen such that:
    * XGB and SVC have great performance for most of the classes, so they should have the biggest weights
    * RF has a really small variance in terms of how well it performs on different classes, but it does not excel
    * The idea is to have RF be a tie-breaker between XGB and SVC
* Hyperparameters were tuned using $5$-fold stratified CV and random search on the parameters

In [36]:
def create_clf():
    from sklearn.ensemble import RandomForestClassifier, VotingClassifier
    from xgboost import XGBClassifier
    from sklearn.svm import SVC
    xgb = XGBClassifier(tree_method='hist',
                        objective='multi:softprob', 
                        num_class=NUM_USERS,
                        nthread=1, n_estimators=100,
                        # hyperparameters follow
                        subsample=0.7,
                        reg_lambda=0.1,
                        min_child_weight=5.0,
                        max_depth=4,
                        learning_rate=0.01,
                        gamma=0.25,
                        colsample_bytree=0.9,
                        colsample_bylevel=0.5)
    rf = RandomForestClassifier(n_estimators=200, random_state=1)
    svc = SVC(C=100, gamma=0.01, decision_function_shape='ovo', 
              max_iter=-1, cache_size=2000, probability=True)
    clf = VotingClassifier(estimators=[('rf', rf), ('xgb', xgb), ('svc', svc)], 
                           voting='soft', weights=[1, 10, 10])
    return clf

Data is transformed using a min-max scaler. From my tests, it performed the best and helped improve accuracy by $\approx 0.5\%$. The others performed similarly, besides the `L1/L2` regularizations which performed worse by $\approx 1\%$.

In [30]:
def create_pipeline():
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import MinMaxScaler
    
    return Pipeline([
        ('scaler', MinMaxScaler()),
        ('clf', create_clf())
    ])

In [ ]:
def hyperparameter_search(x: np.ndarray, y: np.ndarray) -> None:
    from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
    from xgboost import XGBClassifier
    from sklearn.ensemble import RandomForestClassifier, VotingClassifier
    from sklearn.svm import SVC
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import MinMaxScaler
    
    rf = RandomForestClassifier(n_estimators=200, random_state=1)
    xgb = XGBClassifier(tree_method='hist',
                        objective='multi:softprob', 
                        num_class=NUM_USERS,
                        nthread=1, n_estimators=100)
    svc = SVC(decision_function_shape='ovo', max_iter=1000, cache_size=2000, probability=True)
    clf = VotingClassifier(estimators=[('rf', rf), ('xgb', xgb), ('svc', svc)], voting='soft')
    
    pipeline = Pipeline([
        ('scaler', MinMaxScaler()),
        ('clf', clf)
    ])
    
    folds = 5
    param_comb = 40
    params = {
        'clf__xgb__max_depth': [3, 4, 5],
        'clf__xgb__learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'clf__xgb__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'clf__xgb__colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'clf__xgb__colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'clf__xgb__min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'clf__xgb__gamma': [0, 0.25, 0.5, 1.0],
        'clf__xgb__reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'clf__svc__C': [0.1, 1, 10, 100, 1000],
        'clf__svc__gamma': [0.01, 0.1, 1, 10],
        'clf__weights': [[1, 2, 2], [1, 5, 5], [1, 10, 10]],
    }

    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)
    random_search = RandomizedSearchCV(pipeline, param_distributions=params, 
                                       n_iter=param_comb, 
                                       n_jobs=-1, cv=skf.split(x, y), verbose=3, 
                                       random_state=SEED)
    random_search.fit(x, y)
    with open('estimator.txt', 'w') as f:
        print(random_search.best_params_, random_search.best_score_, file=f)

In [26]:
def k_fold_validation(x: np.ndarray, y: np.ndarray, k: int) -> Iterator[Tuple[np.ndarray, np.ndarray]]:
    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=k)
    for train, test in skf.split(x, y):
        x_train, x_test = x[train], x[test]
        y_train, y_test = y[train], y[test]
        
        pipeline = create_pipeline()
        pipeline.fit(x_train, y_train)
        yield y_test, pipeline.predict(x_test)

In [40]:
def print_k_fold_statistics(x: np.ndarray, y: np.ndarray, k: int=3) -> None:
    from sklearn.metrics import classification_report, confusion_matrix
    for i, (y_true, y_predict) in enumerate(k_fold_validation(x, y, k)):
        print(confusion_matrix(y_true, y_predict))
        print(classification_report(y_true, y_predict))
        print('=' * 80)

In [41]:
print_k_fold_statistics(x_train, y_train)

[[144   0   0   0   0   0   5   0   0   1   0   0   0   0   0   0   0   0
    0   0]
 [  0 137   0   3   0   1   0   0   0   2   0   0   0   0   7   0   0   0
    0   0]
 [  0   0 143   0   1   0   0   0   0   0   0   0   6   0   0   0   0   0
    0   0]
 [  0   3   0 144   0   2   0   0   0   0   1   0   0   0   0   0   0   0
    0   0]
 [  0   0   1   0 146   0   0   0   1   0   0   0   2   0   0   0   0   0
    0   0]
 [  0   1   0   0   0 147   0   0   0   1   0   0   0   0   1   0   0   0
    0   0]
 [  4   0   0   0   0   0 145   0   0   0   0   0   0   0   0   0   0   0
    0   1]
 [  0   0   0   0   0   0   0 147   0   0   0   0   0   3   0   0   0   0
    0   0]
 [  0   0   2   0   2   0   0   0 144   0   0   0   2   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0 149   0   0   0   0   1   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 150   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0   0 150   0   0   0   0

In [37]:
def make_submission(x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray) -> pd.DataFrame:
    pipeline = create_pipeline()
    pipeline.fit(x_train, y_train)
    
    result = pd.read_csv(INPUT_PATH + 'sample_submission.csv')
    result['class'] = [int(r) + 1 for r in pipeline.predict(x_test)]
    return result

In [38]:
def get_download_link(df: pd.DataFrame):
    from IPython.display import HTML
    import base64
    
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title='Download predictions',filename='predictions.csv')
    return HTML(html)

In [39]:
get_download_link(make_submission(x_train, y_train, filtered_test_features.values))